In [193]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve

In [194]:
def csr_matvec(val, col_ind, row_ptr, x):
  n = len(row_ptr)-1
  y = np.zeros(n, dtype=x.dtype)
  for i in range(n):
    row_start = row_ptr[i]
    row_end = row_ptr[i+1]
    for k in range(row_start, row_end):
      y[i] += val[k] * x[col_ind[k]]
  return y

def solve_GC(val, col_ind, row_ptr, b, max_iter=1000, tol=1e-10):
  n = len(b)
  x = np.zeros(n)  # Chute inicial
  r = b - csr_matvec(val, col_ind, row_ptr, x)  # Resíduo inicial
  p = r.copy()  # Direção de busca inicial
  rs_old = np.dot(r, r)  # Norma do resíduo ao quadrado

  # Verificação inicial de convergência
  if np.sqrt(rs_old) < tol:
      return x

  for k in range(max_iter):
    Ap = csr_matvec(val, col_ind, row_ptr, p)  # Produto matriz-vetor
    alpha = rs_old / np.dot(p, Ap)  # Tamanho do passo
    x = x + alpha * p  # Atualiza a solução
    r = r - alpha * Ap  # Atualiza o resíduo
    rs_new = np.dot(r, r)  # Nova norma do resíduo

    # Critério de parada
    if np.sqrt(rs_new) < tol:
      break

    # Atualiza a direção de busca
    beta = rs_new / rs_old
    p = r + beta * p
    rs_old = rs_new

  return x

In [195]:
# Coeficiente de difusão q(u) = 1 + u^2
def q_func(u):
    return 1.0 + u**2

# Vetor da solução exata u(x,y) = sin(πx)sin(πy)
def calculate_exact_solution(N, h):
    
    exact_u = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
            x = (i + 1) * h
            y = (j + 1) * h
            exact_u[i, j] = np.sin(np.pi * x) * np.sin(np.pi * y)
    
    return exact_u.flatten()

# Compara a solução numérica com a exata
def calculate_error(numerical_U, exact_U, h):
    
    error_vector = numerical_U - exact_U
    l2_error = np.sqrt(h**2 * np.sum(error_vector**2))
    max_error = np.max(np.abs(error_vector))
    
    return l2_error, max_error

# Cálculo do vetor do sistema não linear F(U)
def calculate_F(U_vector, N, h, source_f):
    
    u = np.zeros((N + 2, N + 2))
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    F = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            r, c = i + 1, j + 1
            
            u_c = u[r, c]
            u_n = u[r+1,c]
            u_s = u[r-1,c]
            u_e = u[r,c+1]
            u_w = u[r,c-1]
            
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            q_ip12j = (q_e + q_c) / 2.0
            q_im12j = (q_w + q_c) / 2.0
            q_ijp12 = (q_n + q_c) / 2.0
            q_ijm12 = (q_s + q_c) / 2.0

            x_term = q_ip12j * (u_e - u_c) - q_im12j * (u_c - u_w)
            y_term = q_ijp12 * (u_n - u_c) - q_ijm12 * (u_c - u_s)
            
            f_ij = source_f[i * N + j]
            F[i, j] = (x_term + y_term) / h**2 - f_ij
            
    return F.flatten()

# Monta a matriz Jacobiana J = dF/dU no formato CSR
def calculate_jacobian_csr(U_vector, N, h):
    """
    Calcula a matriz Jacobiana para o sistema não linear F(U) = 0 no formato CSR.

    Esta versão corrigida inclui a derivada da função fonte f(u, x, y),
    que depende da solução u.

    Args:
        U_vector (np.array): Vetor contendo os valores de u nos pontos internos da malha.
        N (int): Número de pontos internos em cada direção da malha.
        h (float): Espaçamento da malha (h = 1 / (N + 1)).

    Returns:
        tuple: Uma tupla contendo (values, column_indices, row_pointers) para a matriz CSR.
    """
    # Cria uma matriz (N+2)x(N+2) preenchida com zeros para incluir as condições de contorno
    u = np.zeros((N + 2, N + 2))
    # Preenche os pontos internos da matriz com os valores do vetor U
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    
    values = []
    column_indices = []
    row_pointers = [0]

    # Itera sobre cada ponto interno (i, j) da malha para montar uma linha da Jacobiana
    for i in range(N):
        for j in range(N):
            # Mapeia os índices (i, j) do loop para as coordenadas (r, c) na matriz com fronteiras
            r, c = i + 1, j + 1
            
            # Obtém os valores de u no ponto central e nos seus quatro vizinhos
            u_c = u[r, c]      # Central: u_{i,j}
            u_n = u[r + 1, c]  # Norte:   u_{i+1,j}
            u_s = u[r - 1, c]  # Sul:     u_{i-1,j}
            u_e = u[r, c + 1]  # Leste:   u_{i,j+1}
            u_w = u[r, c - 1]  # Oeste:   u_{i,j-1}
            
            # Calcula o coeficiente q(u) para cada ponto
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            # --- Cálculo das derivadas parciais (elementos da Jacobiana) ---

            # Derivada em relação a u_{i,j-1} (Oeste)
            if j > 0:
                val = (q_w + q_c) / 2.0 - u_w * (u_c - u_w)
                column_indices.append(i * N + (j - 1))
                values.append(val / h**2)

            # Derivada em relação a u_{i-1,j} (Sul)
            if i > 0:
                val = (q_s + q_c) / 2.0 - u_s * (u_c - u_s)
                column_indices.append((i - 1) * N + j)
                values.append(val / h**2)
            
            # Derivada em relação a u_{i,j} (Central - Diagonal Principal)
            # 1. Parte do operador diferencial (código original)
            op_deriv = (-(q_e + q_w + q_n + q_s) / 2.0 - 2 * q_c) + u_c * (u_e + u_w + u_n + u_s - 4 * u_c)
            
            # 2. **NOVO**: Parte da derivada da função fonte f(u, x, y)
            x = c * h
            y = r * h
            
            # f(x,y) = 2*pi^2*u*(1+u^2) - 2*u*[pi^2*cos^2(pi*x)*sin^2(pi*y) + pi^2*sin^2(pi*x)*cos^2(pi*y)]
            # df/du = 2*pi^2*(1+3u^2) - 2*[pi^2*cos^2(pi*x)*sin^2(pi*y) + pi^2*sin^2(pi*x)*cos^2(pi*y)]
            
            g_ij = np.pi**2 * (np.cos(np.pi * x)**2 * np.sin(np.pi * y)**2 + 
                               np.sin(np.pi * x)**2 * np.cos(np.pi * y)**2)
            
            f_deriv = 2 * np.pi**2 * (1 + 3 * u_c**2) - 2 * g_ij
            
            # 3. Combina as duas partes
            # Lembre-se que F = Operador - f, então J = J_operador - J_f
            val_c = op_deriv / h**2 - f_deriv
            
            column_indices.append(i * N + j)
            values.append(val_c)

            # Derivada em relação a u_{i+1,j} (Norte)
            if i < N - 1:
                val = (q_n + q_c) / 2.0 - u_n * (u_c - u_n)
                column_indices.append((i + 1) * N + j)
                values.append(val / h**2)

            # Derivada em relação a u_{i,j+1} (Leste)
            if j < N - 1:
                val = (q_e + q_c) / 2.0 - u_e * (u_c - u_e)
                column_indices.append(i * N + (j + 1))
                values.append(val / h**2)
            
            # Atualiza o ponteiro de linha para o formato CSR
            row_pointers.append(len(values))
            
    return np.array(values), np.array(column_indices), np.array(row_pointers)

# Resolve o sistema F(U)=0 com o Método de Newton
def solve_newton(initial_U, N, h, source_f, tol=1e-10, max_iter=50):
    U_k = np.copy(initial_U) 
    print("Iniciando Método de Newton...")
    for k in range(max_iter):
        F_k = calculate_F(U_k, N, h, source_f)
        J_values, J_cols, J_pointers = calculate_jacobian_csr(U_k, N, h)

        s_k = solve_GC(J_values, J_cols, J_pointers, -F_k)

        U_k = U_k + s_k

        step_norm = np.linalg.norm(s_k, np.inf)
        # print(f"Iteração {k+1:2d}: Norma do passo = {step_norm:.2e}")

        if step_norm < tol:
            print(f"\nConvergência atingida em {k+1} iterações.")
            return U_k
    print("\nMáximo de iterações excedido.")
    return U_k

In [ ]:
def solve_broyden(initial_U, N, h, source_f, tol=1e-6, max_iter=50):
    """
    Solve the Poisson equation using Broyden's method.
    """
    J_values, J_cols, J_pointers = calculate_jacobian_csr(initial_U, N, h)
    U_k = np.copy(initial_U)
    A_inv = np.zeros((N * N, N * N))
    for i in range(len(J_pointers) - 1):
        for j in range(J_pointers[i], J_pointers[i + 1]):
            value = J_values[j]
            column = J_cols[j]
            A_inv[i, column] = value
    A_inv = np.linalg.inv(A_inv)
    v = calculate_F(initial_U, N, h, source_f)
    for k in range(max_iter):
        # Compute step
        s_k = -A_inv @ v
        
        # Update solution
        U_k1 = U_k + s_k
        
        # Calculate new residual
        v1 = calculate_F(U_k1, N, h, source_f)
        y_k = v1 - v
        
        # Broyden update of the Jacobian inverse approximation
        denominator = s_k @ s_k
        if np.abs(denominator) < 1e-12:  # Prevent division by zero
            break
            
        A_inv = A_inv + np.outer((s_k - A_inv @ y_k), s_k) @ A_inv / denominator
        
        # Update for next iteration
        U_k = U_k1
        v = v1
        
        # Check convergence
        step_norm = np.linalg.norm(s_k, np.inf)
        residual_norm = np.linalg.norm(v, np.inf)
        # print(f"Iteração {k+1:2d}: Norma do passo = {step_norm:.2e}, Resíduo = {residual_norm:.2e}")
        
        if step_norm < tol:
            print(f"\nConvergência atingida em {k+1} iterações.")
            return U_k  # Return in original shape
    
    print("Atenção: Máximo de iterações atingido sem convergência.")
    return U_k

In [197]:
N = 3
h = 1.0 / (N + 1)  # Grid spacing
source_f = np.array([2 * np.pi**2 * np.sin(np.pi * (i + 1) * h) * np.sin(np.pi * (j + 1) * h) 
                     for i in range(N) for j in range(N)])  # Fonte f(x,y)
initial_U = np.zeros((N, N)).flatten()  # Initial guess (zero everywhere)
x = solve_broyden(initial_U, N, h, source_f)
x_newton = solve_newton(initial_U, N, h, source_f)
print(f"Solução broyden: {x}")
print(f"Solução de Newton: {x_newton}")
print(f"Solução exata: {calculate_exact_solution(N, h)}")

Iteração  1: Norma do passo = 5.88e-01, Resíduo = 9.33e+00
Iteração  2: Norma do passo = 2.95e-01, Resíduo = 3.86e+00
Iteração  3: Norma do passo = 6.03e-02, Resíduo = 9.29e-01
Iteração  4: Norma do passo = 9.03e-03, Resíduo = 1.97e-01
Iteração  5: Norma do passo = 1.08e-03, Resíduo = 1.59e-02
Iteração  6: Norma do passo = 9.73e-05, Resíduo = 3.86e-04
Iteração  7: Norma do passo = 4.49e-06, Resíduo = 1.19e-04
Iteração  8: Norma do passo = 7.42e-07, Resíduo = 3.26e-06

Convergência atingida em 8 iterações.
Iniciando Método de Newton...

Convergência atingida em 38 iterações.
Solução broyden: [-0.46895546 -0.63098143 -0.46895546 -0.63098143 -0.8307232  -0.63098143
 -0.46895546 -0.63098143 -0.46895546]
Solução de Newton: [-0.46895549 -0.63098141 -0.46895549 -0.63098141 -0.83072319 -0.63098141
 -0.46895549 -0.63098141 -0.46895549]
Solução exata: [0.5        0.70710678 0.5        0.70710678 1.         0.70710678
 0.5        0.70710678 0.5       ]
